In [ ]:
# Import necessary libraries
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import numpy as np
from google.colab import files
from PIL import Image

In [ ]:
# Load the Malaria dataset from TensorFlow Datasets
(ds_train, ds_test), ds_info = tfds.load('malaria', 
                                         split=['train[:80%]', 'train[80%:]'],
                                         shuffle_files=True,
                                         as_supervised=True,  # Gets (image, label) pairs
                                         with_info=True)

# Display dataset information
print(ds_info)

# Define image parameters
img_height, img_width = 100, 100
batch_size = 32

# Normalize the images and resize them
def preprocess(image, label):
    image = tf.image.resize(image, [img_height, img_width])
    image = image / 255.0  # Normalize pixel values between 0 and 1
    return image, label

# Apply preprocessing to the datasets
ds_train = ds_train.map(preprocess).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
ds_test = ds_test.map(preprocess).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
# Build the CNN model
model = Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D(),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(2, activation='softmax')  # 2 classes: Parasitized and Uninfected
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(ds_train, validation_data=ds_test, epochs=10)

# Evaluate the model
test_loss, test_acc = model.evaluate(ds_test)
print(f'\nTest accuracy: {test_acc}')

In [ ]:
# Function to plot training history
def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, 'b', label='Training accuracy')
    plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

    plt.show()

plot_history(history)

In [ ]:
# Function to upload and predict
def upload_and_predict(model):
    uploaded = files.upload()
    for filename in uploaded.keys():
        # Load the image
        img = Image.open(filename)
        img = img.resize((img_height, img_width))  # Resize image to match model input shape
        img_array = np.array(img) / 255.0  # Normalize the image
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
        
        # Predict
        predictions = model.predict(img_array)
        score = tf.nn.softmax(predictions[0])
        
        class_names = ['Parasitized', 'Uninfected']
        predicted_class = class_names[np.argmax(score)]
        confidence = 100 * np.max(score)
        
        print(f"Predicted Class: {predicted_class} with {confidence:.2f}% confidence.")

# Call the function to upload and predict
upload_and_predict(model)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# Save the model to your Google Drive
model.save('/content/drive/MyDrive/malaria_model.h5')

In [ ]:
# Save only the weights
model.save_weights('/content/drive/MyDrive/malaria_weights.weights.h5')